In [22]:
class Node:
  def __init__(self, ID, PARENT_ID, deaths, survivors, chi_sq_death, chi_sq_surv, query):
      self.ID = ID #int
      self.PARENT_ID = PARENT_ID #int
      self.deaths = deaths #int
      self.survivors = survivors #int
      self.chi_sq_death = chi_sq_death #float
      self.chi_sq_surv = chi_sq_surv #float
      self.query = query #str
      self.children = [] #list of int. Contiene todos los IDs de los hijos


In [23]:
import numpy as np
import os
from scipy.stats import chi2_contingency, chi2, chisquare
import re
#from node import Node

class Utils:

    def __init__(self):
        self.nodes = []
        self.death_query = 'RESULT == 1'
        self.surv_query = 'RESULT == 0'


    def add_child_to_parent(self,parent_id, child_id):
        i = 0
        found = False
        while i < len(self.nodes) and not found:  # Busca el nodo padre
            if parent_id is self.nodes[i].ID:
                self.nodes[i].children.append(child_id)
                found = True
            i += 1

    def get_node(self,ID):
        i = -1
        while i < len(self.nodes):
            if ID is self.nodes[i].ID:
                return self.nodes[i]
            else:
                i += 1

    # Devuelve número de fallecimientos y supervivencias de ese nodo.
    def get_ds_node(self,ID):
        found = False
        index = 0
        i = 0
        while i < len(self.nodes) and not found:
            if self.nodes[i].ID is ID:
                index = i
                found = True
            i += 1
        return self.nodes[index].deaths, self.nodes[index].survivors

    # Calcula los coeficientes de chi-square usando los valores de muertes y
    # supervivencias del nodo en cuestión y del nodo padre
    def calc_chisq_node(self,parent_d, parent_s, deaths, survivors):
        epsilon_exp = 0.0001  # Coeficiente usado para evitar que la bondad de ajuste sea 0
        # Calculamos las bondades de ajuste
        expected_d = parent_d * (deaths + survivors) / (parent_d + parent_s)
        expected_s = parent_s * (deaths + survivors) / (parent_d + parent_s)
        if expected_d == 0:
            expected_d = epsilon_exp
        if expected_s == 0:
            expected_s = epsilon_exp
        # Los valores de chi-square
        chisq_d = ((deaths - expected_d) ** 2) / expected_d
        chisq_s = ((survivors - expected_s) ** 2) / expected_s

        return chisq_d, chisq_s

    # coefficient:float Coeficiente entre 0 y 1 usado para obtener un % de las características más importantes.
    # columns: Lista de los nombres de las columnas del dataset.
    # importances: Valor de importancia asociado a cada característica en el modelo entrenado.
    def get_top_important_features_list(self,coefficient, columns, importances):
        '''Obtiene las características más importantes en orden descendente'''
        indices = np.argsort(importances)[::-1]  # Indices de las características mas significativas
        mif = importances[indices[0]]  # Valor de la característica más importante
        features = [columns[x] for x in indices if importances[x] >= mif * (1 - coefficient)]
        return features


    def has_pattern(self, matrix):
        matrix[matrix == 0] = 0.0001
        # print('Matrix is:',matrix)
        stat, p, dof, expected = chi2_contingency(matrix, correction=False)
        probability = 0.95
        critical = chi2.ppf(probability, dof)
        if abs(stat) >= critical:
            return True
        else:
            return False


    def obtain_patterns(self):

        patterns = set()
        visited_nodes = []  #Lista auxiliar para guardar los IDs de los nodos que ya han sido visitados.
        # Visita todos los nodos, y de aquellos que no sean el nodo principal y que tengan hijos, obtiene el chi-square de los hijos de ese nodo.
        for node in self.nodes:
            if node.PARENT_ID is not None:
                parent_node = self.get_node(node.PARENT_ID)
                if parent_node.ID not in visited_nodes:  # Evita que el nodo padre sea contado múltiples veces, ya que se calcula el chi-square de los hijos 1 sola vez
                    visited_nodes.append(parent_node.ID)
                    children = parent_node.children  # Obtiene la lista de IDs de sus nodos hermanos
                    if len(children) > 0:  # En el caso de que ese nodo no sea un nodo hoja (Recordar que solo busco nodos padre para calcular el chi-square del total de sus hijos)
                        aux_matrix = []
                        aux_query = ''
                        for child_id in children:  # Access every single sibling node
                            aux_node = self.get_node(child_id)
                            aux_matrix.append([aux_node.survivors, aux_node.deaths])
                            # Se parsea la query para representar el nivel.
                            aux_query = re.search("(.*) ==.*", aux_node.query)
                        #if has_enough_cases:  # Se calcula el p valor de los hermanos en ese subnivel
                        np_matrix = np.array(aux_matrix).astype(float)
                        np_matrix = np_matrix.transpose()
                        has_pattern = self.has_pattern(np_matrix)
                        if has_pattern:
                            patterns.add(aux_query[1])  # Si se encuentra una regla que puede tener un patrón, se incluye.
        return patterns



    def categorize_patterns(self, test_data, patterns, coefficient):
        death_patterns = []
        surv_patterns = []
        for i in range(0, len(patterns)): #Checks all combinations found and checks for both 0 and 1 in the last pathology to study both cases.
            values = [0, 1]
            for j in values:
                query = patterns[i] + ' == ' + str(j) #Adds the 0 or 1 to the last pathology
                deaths = len(test_data.query(query + ' & RESULT == 1'))
                survs = len(test_data.query(query + ' & RESULT == 0'))
                if survs+deaths > 0: #If this pattern has existing cases in total in the training set, is included.
                    if (deaths / (survs + deaths)) >= coefficient: #Checks if the combinations show a pattern for death/surv
                        death_patterns.append([patterns[i] + ' == ' + str(j), deaths, survs + deaths])
                    elif (survs / (survs + deaths)) >= coefficient:
                        surv_patterns.append([patterns[i] + ' == ' + str(j),survs,survs + deaths])
        return death_patterns, surv_patterns

    # Función recursiva encargada de generar el árbol de nodos con sus respectivas queries y obtener en cada nodo la query y el número de fallecimientos y supervivencias de cada uno.
    # features: list of str. Contiene todas las características anteriormente seleccionadas
    # query: str. Variable auxiliar en la que se irá anexando las características junto con sus valores para generar las queries.
    # node_value: int. Representa el valor de la característica en ese nodo en concreto.
    # feature_index: int. índice auxiliar de la lista de características
    # dataset: Pandas DataFrame. Dataset con las filas para obtener el número de fallecimientos y defunciones usando cada query.
    def binary_tree_generator(self,features, dataset, query='', node_value=0, feature_index=0, parent_id=-1):
        # En el caso de que queden características para ampliar el nivel:
        if feature_index < len(features):

            # Caso base para el que se considera como nodo padre de todos.
            if parent_id == -1:
                self.nodes.append(Node(ID=0,
                                  PARENT_ID=None,
                                  deaths=len(dataset.query(self.death_query)),  # Guarda la cantidad de muertes totales.
                                  survivors=len(dataset.query(self.surv_query)),
                                  # Guarda la cantidad de supervivientes totales.
                                  chi_sq_death=0,  # El padre no tiene chi_sq, osea que se deja por defecto a 0.
                                  chi_sq_surv=0,
                                  query=None,  # El padre tampoco tiene query, osea que se deja a 0.
                                  ))
                # Una vez creado el padre, se accede a la primera característica, que representaría el primer nivel.
                # Por cada posible valor que pueda tomar esa característica, se crea un hijo nodo de manera recursiva
                for i in dataset[features[0]].unique():
                    self.binary_tree_generator(features,dataset, parent_id=0, node_value=i)

            # Caso en el que el padre ya ha sido creado
            else:
                # A la query auxiliar se le incluye la característica del nodo junto con el valor asignado
                # tal que queda como FEATURE == X
                aux_query = features[feature_index] + ' == ' + str(node_value)

                # Para poder anexar las características, hay que incluir '&' entre cada valor, pero se puede dar el caso
                # en el que la query esté vacía y poner un '&' no funcionaría, por lo que hay 2 casos posibles:
                # -Query inicial(Primer nivel de hijos).
                # -Subniveles con queries no nulas.
                if query != '':
                    aux_query = query + " & " + aux_query
                else:
                    aux_query = query + aux_query

                # Se obtienen cantidad de supervivencias y defunciones de ese nodo en concreto ejecutando las queries en el dataset.
                deaths = len(dataset.query(self.death_query + ' & ' + aux_query))
                survivors = len(dataset.query(self.surv_query + ' & ' + aux_query))
                # Los valores de muertes y supervivencias del padre se obtienen para calcular el chi-square
                parent_d, parent_s = self.get_ds_node(parent_id)
                aux_d, aux_s = self.calc_chisq_node(parent_d, parent_s, deaths, survivors)

                # Si el nodo se considera que no tiene los casos suficientes, es descartado y el árbol no continúa en esa rama.
                if (deaths + survivors) >= 3:
                    # Se le asigna la ID al nodo como la siguiente a la última utilizada.
                    node_ID = self.nodes[-1].ID + 1
                    self.nodes.append(Node(ID=node_ID,
                                      PARENT_ID=parent_id,
                                      deaths=deaths,
                                      survivors=survivors,
                                      chi_sq_death=aux_d,
                                      chi_sq_surv=aux_s,
                                      query=aux_query,
                                      ))

                    self.add_child_to_parent(parent_id, node_ID)  # La ID del nodo es incluida en la lista de hijos del padre.

                    new_parent_id = self.nodes[-1].ID  # Este nodo ahora hará de padre.
                    for i in dataset[features[feature_index]].unique():
                        new_feature_index = feature_index + 1
                        self.binary_tree_generator(features, dataset, query=aux_query, node_value=i,
                                              feature_index=new_feature_index, parent_id=new_parent_id)



    def writereport(self, route, avg_tr_dth, avg_te_dth, avg_tr_srv, avg_te_srv, prec, rec, acc):

        '''Escribe los coeficientes de predicción del modelo en un fichero.txt, además de los valores de precission, recall y accuracy del modelo de esa iteración.'''
        comb_header = 'Pathologies combinations results:\n'
        death_line = 'Correct predicted casualties in training:' + str(
            avg_tr_dth) + ' || Correct predicted casualties in test:' + str(avg_te_dth) + '\n'
        surv_line = 'Correct predicted csurvivors in training:' + str(
            avg_tr_srv) + ' || Correct predicted survivors in test:' + str(avg_te_srv) + '\n'
        model_header = 'Prediction model results:\n'
        prec_line = "Precission Test: " + str(prec) + '\n'
        rec_line = "Recall Test: " + str(rec) + '\n'
        acc_line = "Accuracy Test: " + str(acc) + '\n'
        f = open(route, 'w')
        f.write(comb_header)
        f.write(death_line)
        f.write(surv_line)
        f.write(model_header)
        f.write(prec_line)
        f.write(rec_line)
        f.write(acc_line)
        f.close()


In [24]:
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, make_scorer, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
#from utils import Utils
import csv


# Index dataset used to retrieve the 10% in each iteration for testing.
# index_dataset = data



def obtain_results(data):

    PROCESS_RUNS = 10
    TEST_SIZE = 0.1
    test_size = int(len(data) * TEST_SIZE)
    bestaccuracy = 0
    obtainedpatterns = set()

    utils = None

    custom_scorer = make_scorer(accuracy_score, greater_is_better=True)
    param_grid = {'n_estimators': [50, 100, 150, 200, 250, 300],  # being the number of trees in the forest.
                  'min_samples_leaf': [3], # number of minimum samples required at a leaf node.
                  'min_samples_split': [6], # number of minimum samples required to split an internal node.
                  'criterion': ['entropy'], # measures the quality of a split. Can use gini's impurity or entropy.
                  }
    clf = GridSearchCV(
        # Evaluates the performance of different groups of parameters for a model based on cross-validation.
        RandomForestClassifier(class_weight='balanced', bootstrap=False),
        param_grid,  # dict of parameters.
        cv=10,  # Specified number of folds in the Cross-Validation(K-Fold).
        scoring=custom_scorer,
        error_score='raise')

    test_data = data.sample(n=test_size, random_state=1)
    data.drop(test_data.index,inplace=True)

    print('Filas de entrenamiento:',len(data),'Filas de testing:',len(test_data))


    for process_id in range(0, PROCESS_RUNS):
        train_data = data.sample(n=int(len(data)*0.6), random_state=1)
        y_aux = train_data['RESULT']
        x_aux = train_data.drop(['RESULT'], axis=1)
        oversample = RandomOverSampler(sampling_strategy='minority')
        x_over, y_over = oversample.fit_resample(x_aux, y_aux)
        train_data = pd.DataFrame(x_over, columns=x_over.columns)
        train_data['RESULT'] = y_over

        #Preparamos los datos para generar el modelo ML:
        dfY = train_data['RESULT']
        dfX = train_data.drop(['RESULT'], axis=1)
        #print('GridSearchCV en proceso...')
        #Se usa GridSearch para obtener la seleccion de parametros que mejores resultados obtiene.
        clf.fit(dfX, dfY)
        # print("Best estimator found by grid search:")
        # print(clf.best_estimator_)
        model = clf.best_estimator_
        utils = Utils()

        # Estimator that was chosen by the search, i.e. estimator which gave highest score (or smallest loss if specified) on the left out data        model.fit(X_train, y_train)

        importances = model.feature_importances_
        #List of top % important features in the model are obtained. This % regulated by coefficient between [0,1].
        features = utils.get_top_important_features_list(coefficient=0.85,
                                                         columns=dfX.columns,
                                                         importances=importances)
        #Genera el árbol binario y obtiene las combinaciones que indican que hay un patrón:
        utils.binary_tree_generator(features, dataset=train_data)
        obtainedpatterns = set.union(obtainedpatterns,utils.obtain_patterns())

        #Se obtiene el accuracy del modelo RandomForest entrenado y se guarda el valor de accuracy más alto de todas las iteraciones.
        y_pred = model.predict(test_data.drop(['RESULT'], axis=1))
        accuracy = metrics.accuracy_score(test_data['RESULT'], y_pred)
        if accuracy > bestaccuracy:
            bestaccuracy = accuracy

    #Divide las combinaciones en si el patrón lo muestran en supervivencia o fallecimiento.
    casualties_categorized_rules, survivors_categorized_rules = \
        utils.categorize_patterns(test_data, list(obtainedpatterns), 0.9) #Returns death and survival patterns where there's a % of the total cases of that specific type.



    casualties_acc = 0
    casualties_total = 0

    survival_acc = 0
    survival_total = 0


    print('-----------------------------')
    print('-----------------------------')
    print('-----------------------------')
    print('-----------------------------')
    print(f'casualties_categorized_rules {len(casualties_categorized_rules)}')
    for pattern in casualties_categorized_rules:
        print(pattern)
        casualties_acc += pattern[1]
        casualties_total += pattern[2]

    print('-----------------------------')
    print('-----------------------------')
    print('-----------------------------')
    print('-----------------------------')
    print(f'survivors_categorized_rules {len(survivors_categorized_rules)}')
    for pattern in survivors_categorized_rules:
        print(pattern)
        survival_acc += pattern[1]
        survival_total += pattern[2]

    print('-----------------------------')
    print('-----------------------------')
    print('-----------------------------')

    with open('combinations_casualties.csv','w',encoding='UTF8') as file:
        writer = csv.writer(file)
        writer.writerow(['Combinacion','Casos_encontrados','Casos_totales'])
        for pattern in casualties_categorized_rules:
            writer.writerow(pattern)
        file.close()

    with open('combinations_survivals.csv','w',encoding='UTF8') as file:
        writer = csv.writer(file)
        writer.writerow(['Combinacion','Casos_encontrados','Casos_totales'])
        for pattern in survivors_categorized_rules:
            writer.writerow(pattern)
        file.close()


    print('Casualties accuracy:',str("{:.2f}".format(100*(casualties_acc/casualties_total)))+'%')
    print('Survival accuracy:',str("{:.2f}".format(100*(survival_acc/survival_total)))+'%')
    print('Top model accuracy:',str("{:.2f}".format(100*bestaccuracy))+'%')



filename = 'clean_dataset'
data_file_name = f'../../data/{filename}.csv'

obtain_results(pd.read_csv(data_file_name))



Filas de entrenamiento: 1570 Filas de testing: 174


/var/folders/m3/06npm5752654l0l9zry_3b0h0000gn/T/ipykernel_5389/56487282.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['RESULT'] = y_over
/var/folders/m3/06npm5752654l0l9zry_3b0h0000gn/T/ipykernel_5389/56487282.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['RESULT'] = y_over
/var/folders/m3/06npm5752654l0l9zry_3b0h0000gn/T/ipykernel_5389/56487282.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

['ANTECEDENTS_PROC_3E0337Z == 1.0 & AGE_HIGHER_60 == 1.0 & AGE_40_60 == 0 & ANTECEDENTS_PROC_3E013NZ == 0 & ANTECEDENTS_PROC_3E013GC == 1.0 & ANTECEDENTS_PROC_3E033NZ == 1.0 & SEXO == 1.0 & ANTECEDENTS_PROC_8E0ZXY6 == 0', 1, 1]
['ANTECEDENTS_PROC_3E0337Z == 0.0 & AGE_HIGHER_60 == 1.0 & AGE_40_60 == 0 & ANTECEDENTS_PROC_3E013NZ == 0 & ANTECEDENTS_PROC_3E013GC == 1.0 & ANTECEDENTS_PROC_3E033NZ == 1.0 & SEXO == 1.0 & ANTECEDENTS_PROC_8E0ZXY6 == 0 & AGE_LOWER_40 == 0.0 & ANTECEDENTS_PROC_3E0F7GC == 0', 1, 1]
['ANTECEDENTS_PROC_3E0337Z == 0.0 & AGE_HIGHER_60 == 1.0 & AGE_40_60 == 0 & ANTECEDENTS_PROC_3E013NZ == 0 & ANTECEDENTS_PROC_3E013GC == 0.0 & ANTECEDENTS_PROC_3E033NZ == 1.0 & ANTECEDENTS_PROC_3E0333Z == 1.0 & SEXO == 1', 2, 2]
['ANTECEDENTS_PROC_3E0337Z == 0.0 & AGE_HIGHER_60 == 1.0 & AGE_40_60 == 0 & ANTECEDENTS_PROC_3E013NZ == 0 & ANTECEDENTS_PROC_3E013GC == 1.0 & ANTECEDENTS_PROC_3E033NZ == 1.0 & SEXO == 1.0 & ANTECEDENTS_DIA_SHORT_I10 == 0 & ANTECEDENTS_PROC_5A1955Z == 0.0 & ANTEC